In [ ]:
import os
import requests

scenarios = ["pc", "rcp85", "rcp26"]
return_periods = [500, 200, 100, 50, 25, 5, 2]

folderr = "workspace/GIRI_raw/"
os.makedirs(folderr, exist_ok=True)

for scenario in scenarios:
    for rp in return_periods:
        url = f"https://hazards-data.unepgrid.ch/global_{scenario}_h{rp}glob.tif"
        filename = os.path.join(folderr, f"flood_{scenario}_{rp}_glob.tif")

        if os.path.exists(filename):
            print(f"[SKIP] {filename} already exists.")
            continue

        print(f"[INFO] Downloading {filename} ...")
        try:
            r = requests.get(url, timeout=60)
            if r.status_code == 200:
                with open(filename, "wb") as f:
                    f.write(r.content)
                print(f"[OK] Saved {filename}")
            else:
                print(f"[WARN] URL not found or unavailable: {url} (status {r.status_code})")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] Failed to download {url}: {e}")

[SKIP] workspace/GIRI_raw/flood_pc_500_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_pc_200_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_pc_100_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_pc_50_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_pc_25_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_pc_5_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_pc_2_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_rcp85_500_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_rcp85_200_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_rcp85_100_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_rcp85_50_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_rcp85_25_glob.tif already exists.
[SKIP] workspace/GIRI_raw/flood_rcp85_5_glob.tif already exists.
[INFO] Downloading workspace/GIRI_raw/flood_rcp85_2_glob.tif ...
[OK] Saved workspace/GIRI_raw/flood_rcp85_2_glob.tif
[INFO] Downloading workspace/GIRI_raw/floo

# setup


In [2]:
import os
import numpy as np
import xarray as xr
import rioxarray as rxr  # make sure: pip/conda install rioxarray

folderr_brazil = "../workspace/GIRI_raw/brazil"

# ---------------- config ----------------
scenarios = ["pc", "rcp85", "rcp26"]
return_periods = [500, 200, 100, 50, 25, 5, 2]


out_nc  = "../workspace/Flood/depth/ensemble_return_period.nc"

# map to human-readable names (as a coord)
dict_scenarios = {
    "pc": "Present Climate",
    "rcp85": "SSP5-8.5",
    "rcp26": "SSP1-2.6",
}

# crop extent (lon_min, lon_max, lat_max, lat_min)
LON_MIN, LON_MAX = -75.0, -30.0
LAT_MAX, LAT_MIN = 10.0, -35.0

# chunk sizes for dask/xarray and NetCDF
CHUNK_LAT, CHUNK_LON = 512, 512

# ----------------------------------------
list_da = []

for scenario in scenarios:
    for rp in return_periods:
        fpath = os.path.join(folderr_brazil, f"flood_{scenario}_{rp}_glob.tif")
        if not os.path.exists(fpath):
            print(f"[WARN] Missing: {fpath} (skipping)")
            continue

        print(f"[INFO] Opening: {fpath}")
        try:
            da = rxr.open_rasterio(
                fpath,
                chunks={"x": CHUNK_LON, "y": CHUNK_LAT},
                masked=True,
            )
        except Exception as e:
            print(f"[ERROR] Failed to open {fpath}: {e}")
            continue

        # squeeze band -> 2D (y,x)
        if "band" in da.dims and da.sizes["band"] == 1:
            da = da.squeeze("band", drop=True)

        # ensure CRS; many global hazard tifs are EPSG:4326
        if da.rio.crs is None:
            da = da.rio.write_crs("EPSG:4326")

        # crop (note: y usually descends; slice handles either order)
        da = da.sel(x=slice(LON_MIN, LON_MAX), y=slice(LAT_MAX, LAT_MIN))

        # rename dims/var
        da = da.rename({"x": "lon", "y": "lat"})
        da.name = "flood_depth"

        # set dtype (float32 is compact and typical for depths)
        da = da.astype("float32")

        # expand coords with scenario + return_period
        da = da.expand_dims(
            {
                "scenario": [dict_scenarios.get(scenario, scenario)],
                "return_period": [rp],
            }
        )

        list_da.append(da)

# guard: anything loaded?
if not list_da:
    raise RuntimeError("No rasters were found/loaded. Check file names and folder.")

# combine by coords into one cube
print("[INFO] Combining arrays…")
da_all = xr.combine_by_coords(list_da, combine_attrs="override")

# order dims
da_all = da_all.transpose("scenario", "return_period", "lat", "lon")

# set some attrs
da_all.attrs.update(
    {
        "long_name": "Flood depth",
        "units": "m",
        "source": "UNEP GRID hazards data",
        "note": "Cropped to South America; scenarios labelled by human-readable names.",
    }
)
da_all["scenario"].attrs["description"] = "Scenario label"
da_all["return_period"].attrs["units"] = "years"

# rechunk uniformly for writing
da_all = da_all.chunk({"scenario": 1, "return_period": 1, "lat": CHUNK_LAT, "lon": CHUNK_LON})

# encode & save
encoding = {
    "flood_depth": {
        "zlib": True,
        "complevel": 4,
        "dtype": "float32",
        "chunksizes": (1, 1, CHUNK_LAT, CHUNK_LON),
        "_FillValue": np.float32(np.nan),  # keep NaN for no-data
    }
}


[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_pc_500_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_pc_200_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_pc_100_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_pc_50_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_pc_25_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_pc_5_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_pc_2_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_rcp85_500_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_rcp85_200_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_rcp85_100_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_rcp85_50_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_rcp85_25_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_rcp85_5_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_rcp85_2_glob.tif
[INFO] Opening: ../workspace/GIRI_raw/brazil/flood_rc

In [ ]:
out_nc  = "../workspace/hazards/Flood/depth/ensemble_return_period.nc"
print(f"[INFO] Writing NetCDF -> {out_nc}")
da_all.to_dataset(name="flood_depth").to_netcdf(out_nc, encoding=encoding)
print("[OK] Done.")
